In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import string
import warnings
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics.pairwise import pairwise_distances 
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [3]:
cd drive/My Drive/Colab Notebooks/rs

/content/drive/My Drive/Colab Notebooks/rs


In [4]:
song_data = pd.read_csv("song_data.txt", sep = ',')
song_data.head()

,user,song,play_count,track_id,artist,title
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1,TRAEHHJ12903CF492F,Dwight Yoakam,You're The One
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1,TRLGMFJ128F4217DBE,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1,TRTNDNE128F1486812,Cartola,Tive Sim
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1,TRASTUE128F930D488,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1,TRFPLWO128F1486B9E,Miguel Calo,El Cuatrero


In [5]:
song_data1 = song_data[['user', 'song', 'play_count', 'title']][:300000]

In [6]:
song_data1['user_id'] = song_data1.groupby(['user']).ngroup()
song_data1['song_id'] = song_data1.groupby(['song']).ngroup()

In [7]:
song_data1.head()

,user,song,play_count,title,user_id,song_id
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1,You're The One,22494,5047
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1,Horn Concerto No. 4 in E flat K495: II. Romanc...,22494,13839
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1,Tive Sim,22494,17647
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1,Catch You Baby (Steve Pitron & Max Sanna Radio...,22494,24963
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1,El Cuatrero,22494,11897


In [8]:
play_counts = pd.DataFrame(song_data1.groupby('title')['play_count'].mean())
play_counts.head()

,play_count
title,
Light Mass Prayers,1.0
Ég Átti Gráa Æsku,31.0
(Jack The Stripper),1.0
I Will Not Reap Destruction,1.8
You,1.0


In [9]:
play_counts['num of play_counts'] = pd.DataFrame(song_data1.groupby('title')['play_count'].count())
play_counts.head()

,play_count,num of play_counts
title,,
Light Mass Prayers,1.0,1
Ég Átti Gráa Æsku,31.0,1
(Jack The Stripper),1.0,1
I Will Not Reap Destruction,1.8,5
You,1.0,1


In [10]:
songmat = song_data1.pivot_table(index='user_id', columns='title', values='play_count')

In [11]:
songmat.head()

title,Light Mass Prayers,Ég Átti Gráa Æsku,(Jack The Stripper),I Will Not Reap Destruction,You,!@*$%#,!Hoy Estreno!,#!*@ You Tonight [Featuring R. Kelly] (Explicit Album Version),#1,#1 Fan [Feat. Keyshia Cole & J. Holiday] (Album Version),#1 Hit Jam,#1 Stunna,#1 Trap Pick,#2 For Prepared Wire-Strung Harp With Tremolo Pedal,#3 For Bray Harp,#4 (Album Version),#40,#69 Bronson Road,$ 1000 Wedding,$ VS. Entertainment,$100 Dollar Bill Y'all (Explicit),$1000 Wedding,$29.00,$300,$87 And A Guilty Consience That Gets Worse The Longer I Go,$in$,$timulus Plan,& Down,&..Over&..Time&.,&And So You Fell,&And The Battle Begun,&And The Day Turned To Night,&And The World Will Cease To Be,' Cello Song,'60 - '65,'94 Ho Draft,'97 Bonnie & Clyde,'A Cimma,'Bout It,'Bout My Paper,...,Älska Mig Som Mest Då,Älä kerro meille,Änglamark,Ännu En Dag,Ännu glöder solen,Är Det Där Vi Är Nu,Å Ha En Venn Som Deg,Årstider,Ætti Mørkna,Ça Marche,Çok Asik,È Non È,É Fogo,É Mágoa,É Papa Ré,Élni Vagy Égni,Época,Époque d'opinions,Ê Boi,Í Gær,Így is jó,Ñapaes,Ñero,Ô_ Meu Pai,Ôdjus Fitxadu (With My Eyes Shut),Ôrô,Ölürüm Sana,Öppna Upp Ditt Fönster,Örökké Tart,Ösforrás,Új esztendo,Új templom épül,Úsame,Über Grenzen Geh'n,Üc,Ünzile,Þau hafa sloppið undan þunga myrkursins,Þú ert jörðin,Þú ert sólin,ça Me Fait Du Bien
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
song1_user_ratings = songmat["You're The One"]
song1_user_ratings.head()

user_id
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: You're The One, dtype: float64

In [13]:
similar_to_song1 = songmat.corrwith(song1_user_ratings)

In [14]:
corr_song1 = pd.DataFrame(similar_to_song1, columns=['correlation'])
corr_song1.dropna(inplace=True)
corr_song1.head()

,correlation
title,
(They Long To Be) Close To You,1.000000
(You Drive Me) Crazy (The Stop Remix!),0.547619
(iii),0.608565
...And I Love You,-1.000000
16 Candles,-0.027266


In [15]:
corr_song1.sort_values('correlation', ascending=False).head(10)

,correlation
title,
(They Long To Be) Close To You,1.0
House Of Broken Love,1.0
I'm So Into You,1.0
I'm In Miami Bitch,1.0
I Used To Have A Best Friend (But Then He Gave Me An STD),1.0
I Go To Sleep,1.0
I Forgot To Be Your Lover,1.0
Howlin For You,1.0
Hollaback Girl,1.0


In [16]:
corr_song1 = corr_song1.join(play_counts['num of play_counts'])
corr_song1.head()

,correlation,num of play_counts
title,,
(They Long To Be) Close To You,1.000000,40
(You Drive Me) Crazy (The Stop Remix!),0.547619,43
(iii),0.608565,67
...And I Love You,-1.000000,5
16 Candles,-0.027266,224


In [17]:
corr_song1[corr_song1['num of play_counts']>100].sort_values('correlation', ascending=False).head()

,correlation,num of play_counts
title,,
Ghosts 'n' Stuff (Original Instrumental Mix),1.000000,140
You're The One,1.000000,918
Heads Will Roll,0.999810,105
The Real Slim Shady,0.980730,137
A-Punk (Album),0.980444,111
